In [1]:
import pandas as pd
import tushare as ts
token = 'your_token'
ts.set_token(token)
pro = ts.pro_api(token)

In [9]:
df = pro.index_daily(ts_code='000001.SH',start_date = '20051210',end_date = '20220630')

In [10]:
df = df.set_index(pd.to_datetime(df['trade_date']))
df.sort_index(inplace = True)
df.drop(columns=['pre_close','change','pct_chg','vol','amount'],inplace=True)

In [11]:
# Z分数标准化
df['Z_close'] = (df['close']-df['close'].rolling(252).mean())/df['close'].rolling(252).std()
df['Z_open'] = (df['open']-df['close'].rolling(252).mean())/df['close'].rolling(252).std()
df['Z_high'] = (df['high']-df['close'].rolling(252).mean())/df['close'].rolling(252).std()
df['Z_low'] = (df['low']-df['close'].rolling(252).mean())/df['close'].rolling(252).std()

In [12]:
df_train = df.loc['20070104':'20161230',:]
df_test = df.loc['20170103':'20220630',:]

In [13]:
df_train[:20]

,ts_code,trade_date,close,open,high,low,Z_close,Z_open,Z_high,Z_low
trade_date,,,,,,,,,,
2007-01-04,000001.SH,20070104,2715.719,2728.188,2847.615,2684.818,3.439970,3.478942,3.852213,3.343389
2007-01-05,000001.SH,20070105,2641.334,2668.577,2685.804,2617.019,3.139624,3.223460,3.276474,3.064798
2007-01-08,000001.SH,20070108,2707.199,2621.068,2708.444,2620.625,3.266033,3.005531,3.269798,3.004191
2007-01-09,000001.SH,20070109,2807.804,2711.049,2809.394,2691.360,3.478914,3.192154,3.483626,3.133801
2007-01-10,000001.SH,20070110,2825.576,2838.113,2841.741,2770.988,3.442539,3.478964,3.489504,3.283939
2007-01-11,000001.SH,20070111,2770.110,2819.367,2841.180,2763.886,3.208129,3.348849,3.411165,3.190348
2007-01-12,000001.SH,20070112,2668.110,2745.321,2782.025,2652.578,2.862928,3.080718,3.184249,2.819117
2007-01-15,000001.SH,20070115,2794.701,2660.070,2795.331,2658.879,3.149964,2.776339,3.151713,2.773033
2007-01-16,000001.SH,20070116,2821.017,2818.663,2830.803,2757.205,3.152615,3.146188,3.179332,2.978402


In [15]:
df_train.to_csv('train.csv')
df_test.to_csv('test.csv')